In [ ]:
import itertools
import numpy as np

# ==== 全基底（64次元） ====
all_basis = [tuple(int(x) for x in format(i, '06b')) for i in range(2**6)]

# ==== 3粒子空間の基底インデックスを求める ====
basis_index = [all_basis.index(b) for b in basis]

# ==== 20次元→64次元への埋め込み ====
H_full = np.zeros((64, 64), dtype=complex)
for i, bi in enumerate(basis_index):
    for j, bj in enumerate(basis_index):
        H_full[bi, bj] = H[i, j]

# ==== パウリ行列定義 ====
I = np.array([[1,0],[0,1]])
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])
paulis = [I, X, Y, Z]
labels = ['I','X','Y','Z']

# ==== Pauli 展開 ====
coeffs = {}
for ops in itertools.product(range(4), repeat=6):
    P = paulis[ops[0]]
    for k in ops[1:]:
        P = np.kron(P, paulis[k])
    c = np.trace(P @ H_full) / 64
    if abs(c) > 1e-8:
        label = ''.join(labels[k] for k in ops)
        coeffs[label] = np.real_if_close(c)

# ==== 出力 ====
print(f"\nNonzero Pauli terms ({len(coeffs)} terms):")
for k, v in sorted(coeffs.items(), key=lambda x: -abs(x[1])):
    print(f"{k:10s} : {v:.6f}")


In [ ]:
qc_prep = QuantumCircuit(num_orbitals)
qc_prep.x(0)
qc_prep.x(1)
qc_prep.x(2)
qc_prep.draw('mpl')

In [ ]:
qc_prep = QuantumCircuit(num_orbitals)
qc_prep.append(Uprep, range(num_orbitals))
state_vector = Statevector.from_instruction(qc_prep)
display(state_vector.draw("latex"))

# 逆演算U^{\dagger}_pを作用させて|0>に戻るかチェック
qc_prep.append(Updag, range(num_orbitals))
state_vector = Statevector.from_instruction(qc_prep)
display(state_vector.draw("latex"))

In [ ]:
from qiskit.quantum_info import SparsePauliOp
import numpy as np

# ==== coeffs は前のコードで作成済みの辞書 ====
# 例: coeffs = {'IIIIII': 1.0, 'ZZIIII': 0.3, ...}

# Pauli文字列と係数をリストに変換
pauli_strings = list(coeffs.keys())
coeff_values  = np.array([coeffs[k] for k in pauli_strings], dtype=float)

# SparsePauliOp を作成
H_op = SparsePauliOp(pauli_strings, coeff_values)

print(H_op)

In [ ]:
trotter_steps = 1 
ancilla_qubits = [0]
target_qubits = list(range(1,num_orbitals+1))

t_i = 0.12
t_j = 0.56
Ui = PauliEvolutionGate(H_op, t_i, synthesis=SuzukiTrotter(order=1,reps=trotter_steps))
Uj = PauliEvolutionGate(H_op, t_j, synthesis=SuzukiTrotter(order=1,reps=trotter_steps))

circuit_Ui = QuantumCircuit(num_orbitals)
circuit_Ui.append(Ui, range(num_orbitals))
circuit_Ui.name = '$U(t_i)$'
gate_Ui = circuit_Ui.to_gate()

circuit_Uj = QuantumCircuit(num_orbitals)
circuit_Uj.append(Uj, range(num_orbitals))
circuit_Uj.name = '$U(t_j)$'
gate_Uj = circuit_Uj.to_gate()

# c-Ui: controlled U = exp(-iHt_i)
circuit_cUi = QuantumCircuit(num_orbitals)
circuit_cUi.append(Uprep, range(num_orbitals))
circuit_cUi.append(Ui, range(num_orbitals))
circuit_cUi.name = 'c-$U(t_i) U_p$'
gate_cUi = circuit_cUi.to_gate().control(1)

# C-Uj 
circuit_cUj = QuantumCircuit(num_orbitals)
circuit_cUj.append(Uprep, range(num_orbitals))
circuit_cUj.append(Uj, range(num_orbitals))
circuit_cUj.name = 'c-$U(t_j) U_p$'
gate_cUj = circuit_cUj.to_gate().control(1)

In [ ]:
qc_psi_i = QuantumCircuit(num_orbitals)
qc_psi_i.append(Uprep, range(num_orbitals))
qc_psi_i.append(gate_Ui, range(num_orbitals))
psi_i = Statevector.from_instruction(qc_psi_i)

qc_psi_j = QuantumCircuit(num_orbitals)
qc_psi_j.append(Uprep, range(num_orbitals))
qc_psi_j.append(gate_Uj, range(num_orbitals))
psi_j = Statevector.from_instruction(qc_psi_j)

overlap = np.dot(psi_i.data.conj(), psi_j.data)
Re_overlap = np.real(overlap)
Im_overlap = np.imag(overlap)

print("psi_i"); display(psi_i.draw("latex"))
print("psi_j"); display(psi_j.draw("latex"))
print("<psi_i|psi_j>", overlap)
print("cos(t_i-t_j)", np.cos( np.sum(coeff_values) * (t_i-t_j) ), "sin(t_i-t_j)",  np.sin( np.sum(coeff_values) * (t_i-t_j) ) )

In [ ]:
# circuit to measure Re <psi_i|psi_j>
qc_re = QuantumCircuit(1+num_orbitals,1)
qc_re.h(0)
qc_re.append(gate_cUi, ancilla_qubits + target_qubits)
qc_re.x(0)
qc_re.append(gate_cUj, ancilla_qubits+target_qubits)
qc_re.h(0)
qc_re.measure(0,0)
display(qc_re.draw('mpl'))
qc_re = qc_re.decompose()

In [ ]:
# circuit to measure Im <psi_i|psi_j>
qc_im = QuantumCircuit(1+num_orbitals,1)
qc_im.h(0)
qc_im.append(gate_cUi, ancilla_qubits + target_qubits)
qc_im.x(0)
qc_im.append(gate_cUj, ancilla_qubits+target_qubits)
qc_im.sdg(0)
qc_im.h(0)
qc_im.measure(0,0)
display(qc_im.draw('mpl'))
qc_im = qc_im.decompose()

In [ ]:
from qiskit_aer.primitives import SamplerV2
sampler = SamplerV2()
num_shot = 10**5

job = sampler.run([qc_re, qc_im], shots=num_shot)
results  = job.result()

# Real part
prob = results[0].data.c.get_counts()
p0 = prob.get('0',0) / num_shot
p1 = prob.get('1',0) / num_shot
print(f"Estimated Re<psi_i|psi_j> = {p0 - p1:9.5f}  Exact: {Re_overlap:9.5f} Error: {np.abs(p0-p1-Re_overlap):6.2e}")

# Imaginary part
prob = results[1].data.c.get_counts()
p0 = prob.get('0',0) / num_shot
p1 = prob.get('1',0) / num_shot
print(f"Estimated Im<psi_i|psi_j> = {p0 - p1:9.5f}  Exact: {Im_overlap:9.5f} Error: {np.abs(p0-p1-Im_overlap):6.2e}")

In [ ]:
def additional_qc(qc_in, pauli_str, register_target, Qiskit_order=True):
    pauli_str = str(pauli_str)
    if Qiskit_order:
        pauli_str = pauli_str[::-1]

    for i in range(len(pauli_str)):
        if pauli_str[i] == "X":
            qc_in.h(register_target[i])
        elif pauli_str[i] == "Y":
            qc_in.sdg(register_target[i]); qc_in.h(register_target[i])
        elif pauli_str[i] == "Z" or pauli_str[i] == "I":
            pass
        else:
            raise ValueError("Invalid Pauli string: ", pauli_str)

def get_idx_to_measure(pauli_str, Qiskit_order=True):
    idxs = [ idx for idx, p in enumerate(pauli_str) if p != "I"]
    if Qiskit_order:
        idxs = [ len(pauli_str) - 1 - idx for idx in idxs]
    return idxs

hamil = ["XXIIIY", "IYIYII"]
for h in hamil:
    idx_relevant = get_idx_to_measure(h)
    print(h) # idx_relevant)
    qc = QuantumCircuit(num_orbitals)
    qc.append(Uprep, range(num_orbitals))
    additional_qc(qc, h, range(num_orbitals))    
    display(qc.draw('mpl', scale=0.3))

In [ ]:
Hamil_coeffs = H_op.coeffs
Hamil_paulis = H_op.paulis

idxs_relevant = [ ]
qcs = [ ]
verbose = False
using_statevector = True
for h_term in list(H_op.paulis):
    h_term = h_term.to_label()
    idx_relevant = get_idx_to_measure(h_term)
    idxs_relevant.append(idx_relevant)
    qc = QuantumCircuit(num_orbitals)
    qc.append(Uprep, range(num_orbitals))
    additional_qc(qc, h_term, range(num_orbitals))
    if verbose:
        print(h_term, idx_relevant)
        display(qc.draw('mpl', scale=0.5))
    qc = qc.decompose().decompose()
    if not(using_statevector):
        qc.measure_all()
    qcs.append(qc)


if using_statevector:
    results = [ Statevector.from_instruction(qc).probabilities_dict() for qc in qcs]
else:
    job = sampler.run(qcs, shots=num_shot)
    results  = job.result()

Esum = 0.0
for idx in range(len(qcs)):
    h_term = H_op.paulis[idx]
    idx_relevant = idxs_relevant[idx]
    if using_statevector:
        res = results[idx]
    else:
        res = results[idx].data.meas.get_counts()

    expval, dummy, dummy_ = expec_Zstring(res, idx_relevant)
    print(h_term.to_label(), Hamil_coeffs[idx], expval)
    Esum += Hamil_coeffs[idx] * expval
print("Esum", Esum, "Egs_exact", Egs_exact)

In [ ]:
initial = "exact"
initial = ""

if initial == "exact":
    Uprep = Uprep # exact ground state
else:
    np.random.seed(0)
    params_random = np.random.rand(5)
    Uprep = Uprep# random state

Hamil_coeffs = H_op.coeffs
Hamil_paulis = H_op.paulis
dt = 1.0

using_statevector = True
#using_statevector = False
Hmat, Nmat, Ens = QuantumKrylov(Uprep, H_op, sampler, delta_t=dt, max_iterations=min([10,len(evecs)]), trotter_steps=1, 
                           num_shot=10**2, 
                           ancilla_qubits=[0], target_qubits=list(range(1,num_orbitals+1)),
                           using_statevector=using_statevector)
print("Eexact", eigvals)
plot_convergence(Ens, eigvals)

if initial == "exact":
    print("<Ψ0|Ψ1>", np.exp(- 1j * Egs_exact * dt) , "N[0,1]", Nmat[0,1])
    print("<Ψ0|H|Ψ0>", Egs_exact, "H[0,0]", Hmat[0,0])
    print("<Ψ0|H|Ψ1>", np.exp(- 1j * Egs_exact * dt) * Egs_exact, "H[0,1]", Hmat[0,1])
    print("<Ψ0|Ψ2>", np.exp(- 1j * Egs_exact * dt * 2), "N[0,2]", Nmat[0,2])
    print("<Ψ0|H|Ψ2>", np.exp(- 1j * Egs_exact * dt * 2) * Egs_exact, "H[0,2]", Hmat[0,2])

